In [1]:
import pandas as pd
import plotly.express as px
import streamlit as st
import matplotlib.pyplot as plt
df= pd.read_csv("history.csv")

In [3]:
df[df['title'].str.contains('Piazza')]

,date,title,url
2,2023-11-01 16:11:49,CSC 2516 | Piazza QA,https://piazza.com/class/lmhqcchw29mpw/post/172
3,2023-11-01 16:11:07,CSC 2516 | Piazza QA,https://piazza.com/class/lmhqcchw29mpw/post/173
4,2023-11-01 16:11:03,CSC 2516 (1 unread) | Piazza QA,https://piazza.com/class/lmhqcchw29mpw/post/174
5,2023-11-01 16:11:01,CSC 2516 (2 unread) | Piazza QA,https://piazza.com/class/lmhqcchw29mpw/post/16
6,2023-11-01 16:10:58,CSC 2516 (1 unread) | Piazza QA,https://piazza.com/class/lmhqcchw29mpw/post/16
...,...,...,...
52170,2023-09-13 13:40:11,CSC 413/2516 (1 unread) | Piazza QA,https://piazza.com/class/lmhqcchw29mpw
52172,2023-09-13 13:38:09,CSC4132516 at University of Toronto | Piazza,https://piazza.com/utoronto.ca/fall2023/csc413...
63995,2023-07-25 04:40:53,ECE1779 at University of Toronto | Piazza,https://piazza.com/utoronto.ca/winter2022/ece1779
71672,2023-07-06 02:46:13,Piazza • Ask. Answer. Explore. Whenever.,https://piazza.com/account/login?message=Pleas...
